In [6]:
import pandas as pd
import psycopg2
from psycopg2 import sql
#set up connection
db_username = 'postgres'
db_password = '123123123'
db_host = '127.0.0.1'
db_port ='5432'
db_name = 'myfirstpostgresqldb'

try:
    conn = psycopg2.connect(
    dbname=db_name,
    user=db_username,
    password=db_password,
    host=db_host,
    port=db_port)
except Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
#connect to curser
try:
    cur=conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the database")
    print(e)


In [12]:
#set automatic committed
conn.set_session(autocommit=True)

#define csv file path
csv_file_path='/Users/issacmiao/Downloads/AmazonSalesData.csv'

df=pd.read_csv(csv_file_path)

# Generate the CREATE TABLE statement
create_table_query = sql.SQL('create table if not exists amazon_sales_data ({})').format(sql.SQL(',').join([sql.Identifier(col) + sql.SQL('text') for col in df.columns]))

try:
    cur.execute(create_table_query)
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
    
# Commit the transaction
#conn.commit()

In [14]:
print(df.head())

                              Region                Country        Item Type  \
0              Australia and Oceania                 Tuvalu        Baby Food   
1  Central America and the Caribbean                Grenada           Cereal   
2                             Europe                 Russia  Office Supplies   
3                 Sub-Saharan Africa  Sao Tome and Principe           Fruits   
4                 Sub-Saharan Africa                 Rwanda  Office Supplies   

  Sales Channel Order Priority Order Date   Order ID  Ship Date  Units Sold  \
0       Offline              H  5/28/2010  669165933  6/27/2010        9925   
1        Online              C  8/22/2012  963881480  9/15/2012        2804   
2       Offline              L   5/2/2014  341417157   5/8/2014        1779   
3        Online              C  6/20/2014  514321792   7/5/2014        8102   
4       Offline              L   2/1/2013  115456712   2/6/2013        5062   

   Unit Price  Unit Cost  Total Revenue  Tot

In [16]:
# Use copy_from() to import the data from the DataFrame to PostgreSQL
from io import StringIO
sio = StringIO()
df.to_csv(sio, header=False, index=False)
sio.seek(0)

cur.copy_from(sio, 'amazon_sales_data', sep=',')
print(f"Data imported successfully into 'amazon_sales_data' table.")


Data imported successfully into 'amazon_sales_data' table.


In [17]:
cur.close()
conn.close()